In [2]:
from __future__ import print_function
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pylab
import pandas as pd
import numpy as np
import os
import sys
import gzip
import itertools
import operator
import subprocess
import twobitreader
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pysam
import shutil

from AAVseq.AAVseq_helpersDanner import *
from AAVseq.AAVseqDanner import *


# Danner_AAVseqLogic

#  This is a simliar to the guideseq pipeline. Different than the oirignal guideseq pipeline but built in a simliar form. 
## It checks for the on target priming, the uses bowtie2 trimming off the adapter seq. Then exports a bed file for comparing to CRISPRgold


#### 1. Check reads were correctly primed.
#### 2 Trim reads 
#### 3. Align the reads against the whole geneome with local alignment. 
#### 4. Alignprocess bam file and pull out the high mapQ reads
#### 5. convert to bed
#### A3. Do global end-to-end alignment
#### A4 global end-to-end
#### A5 convert to bed

It is important to trim R1 and R2 because if it is short then the read oges onto the chip. Also we will use global and local and compare the results because it is hard to trim to the end of the AAV due ot the random nature of hte integration.



We use read1 for checking priming. Also we could use that for mapping (or paired end). On both the 3 and 5' direction the first 34 nucleotides of the read contain the integrated ODN

In [28]:
#Directory

directory = '/home/eric/Data/Spaced_Nicking/AAVseq1_200930_MN00157_0064_A000H37GLC'

#this is a minimal directory of only 300 files for quick debugging
#directory = '/Workspace/Spaced_Nicking/GuideSeq_MiniSeq1/10000files'

print(directory)

/home/eric/Data/Spaced_Nicking/AAVseq1_200930_MN00157_0064_A000H37GLC


### Discard Mispriming Reads
When you put a universal primer on the ends of everything, every mispriming event will amplify. An effect we normally don't deal with. I did nested PCR to reduce this. However 85% of the alignments in the UDITAS data I looked at seemed to be mispriming. They did all their blasting and analysis before removing mispriming. But to save computational power and remove error early on I will discard mispriming events. 
They discard these only for plasmid alignments analyze_alignments_plasmid for some reason which comes from the bam file.

### for guideseq read1 is where the ODN primer binds and so that is how you tell targeting. For the reverse it was binidng to the transposon which is random but then the seq primer binds right on it so the the read for read2 begins in the genomic sequence. the Rev primer could misprime but it shouldnt matter as it would then just bind and make smaller amplicons

In [3]:
#the amplicon info is related to the line on the csv file. It is indexed from 0.
#it is on my extera space so need to make sure that is mounted. (first thing to check if it throws and error)


# THIS IS WHAT RUNS THE PRIMER CHECK THROUGH ALL SAMPLES
#input the sample range or list below


results_df_all = pd.DataFrame()

results_folder = os.path.join(directory, 'results')
if not os.path.exists(results_folder):
    os.mkdir(results_folder)
results_file = os.path.join(directory, 'results','all_priming.xlsx')
    

#make results tree
# 15 samples

for i in range(15):
        
    amplicon_info = get_csv_data(directory, i)

    #5primer is everything but AT, AT is for checking mispriming. The full sequence is the olgio from guideseq.
    #EVERYTHING IS CAPITAL
    
    # only use 3' as the ends are symetrial so put in 3 on the .csv sheet
    guideseq3_seq = 'TCTCTGCGCGCTCGCTCGCTCACTGA'
    guideseq3_primeronly = 'TCTCTGCGCGCTCGCTCG'
    #3primer is everything but the final AC. The AC is for checking mispriming
    guideseq5_seq = ''
    guideseq5_primeronly = ''

    direction = amplicon_info['Direction']
    
    #removed the section choosing between the 5' direction and 3' because it's symmetric
    primer_seq_plus_downstream = guideseq3_seq
    primer_seq = guideseq3_primeronly
    
    df_sample_results = correct_priming_guideseq(directory, amplicon_info, primer_seq, primer_seq_plus_downstream)
    results_df_all = results_df_all.append(df_sample_results, ignore_index=True)
    print('done with sample', i)

print(results_df_all)
results_df_all.to_excel(results_file)    
#print reults to excel

done with sample 0
done with sample 1
done with sample 2
done with sample 3
done with sample 4
done with sample 5
done with sample 6
done with sample 7
done with sample 8
done with sample 9
done with sample 10
done with sample 11
done with sample 12
done with sample 13
done with sample 14
    sample_name    i7    i5  total_reads  reads_with_good_priming  \
0   AAV_seq_1.1  N701  S502       770517                        5   
1   AAV_seq_1.2  N701  S503       639421                        2   
2   AAV_seq_1.3  N701  S505       524043                        3   
3   AAV_seq_2.1  N702  S502       258548                   133559   
4   AAV_seq_2.2  N702  S503       197160                   117796   
5   AAV_seq_2.3  N702  S505       167655                   103758   
6   AAV_seq_3.1  N703  S502       250131                   185015   
7   AAV_seq_3.2  N703  S503       169333                   132932   
8   AAV_seq_3.3  N703  S505       211348                   167826   
9   AAV_seq_4.1  N70

In [15]:
### TRIMMING ####
#need to trim off the end of the short reads. This is for amplicons that were too short and have the other side on them.
# expect more trimmed read 2 and than read 1. Read 2 is longer htan read one as we trim off the ~30 bases of good priming 
# form read1 so sort amplicons get trimmed for read2 before being trimmed for read1.

for i in range(15):
        
    amplicon_info = get_csv_data(directory, i)
    
    trim_AAVseq(directory, amplicon_info)
    
    print('done with sample', i)


done with sample 0
done with sample 1
done with sample 2
done with sample 3
done with sample 4
done with sample 5
done with sample 6
done with sample 7
done with sample 8
done with sample 9
done with sample 10
done with sample 11
done with sample 12
done with sample 13
done with sample 14


In [6]:
# BOWTIE2_INDEXES are needed for global alignments
#check in bash: > ECHO $GENOMES_2BIT
%env BOWTIE2_INDEXES=/home/eric/Data/Ref_Genomes

env: BOWTIE2_INDEXES=/home/eric/Data/Ref_Genomes




## Need to make an new bowtie2 index file that includes targeting for alignment agains the whole genome so it is all in one sheet together. 

### Other option would be to align it to the amplicon, extract unaligned files and then align to the genome but seems cleaner this way. 
#### Ideally every sequence is unique between the targeting vector and genome.


1. Build your fastas of interest and label .fa files.
    1. You need fasta of hg38 or reference genome. You can pull this from downloaded bowtie indexed sampels and then use the following command to turn the index into a fasta file: bowtie2-inspect hg38 > hg38.fa   
    2. Put all the fasta files in the same folder. Should also use the transfected plasmid
2. index the files with bowtie
    1. use the command bowtie2-build -f pE049,pe038_mc.fa,hg38.fa -p hg38_plus_targetvectorandplasmid
    2. this has the -p to make it take less ram in my case.
    3. In this case it adds the hg38 and the minicircle targeting file together
    4. I have a Intel® Core™ i7-5500U CPU @ 2.40GHz × 4 with 15.1 GiB ram and it required about 13.8 gigs of ram and 2 hours to do a hg38+small fasta index
    5. be sure to pay attention to the name of the new indexed file. "hg38_plus_targetvector" in example above. Add it to the sample_info.csv sheet. Under the tab 'genome_plus_targeting'.
    6. you can check it indexed correcctly: bowtie2-inspect -s hg38_plus_targetvector
    

    



## Now we blast the correctly primed sequences.
We don't need to trim off anything. But we should remove the length of the ODN primer sequence before blasting for Read1

In [29]:

def align_guideseq_genome_local(dir_sample, amplicon_info, assembly, single_or_paired = 'single', ncpu=12, keep_sam=0):

    # We first check if the experiment had any guides
    N7 = amplicon_info['index_I1']
    N5 = amplicon_info['index_I2']
    
    #input
    # Read1 for guideSeq they way Chu did it
    file_R1 = create_filename(dir_sample, N7, N5, 'R1trimmed')
    file_R2 = create_filename(dir_sample, N7, N5, 'R2trimmed')
    
    if single_or_paired == 'single':
        file_sam_genome_local = create_filename(dir_sample, N7, N5, 'sam_genome_local_single')
        file_sam_report_genome_local = create_filename(dir_sample, N7, N5, 'sam_report_genome_local_single')
    
        if not os.path.exists(os.path.dirname(file_sam_genome_local)):
            os.mkdir(os.path.dirname(file_sam_genome_local))

        file_bam_genome_local = create_filename(dir_sample, N7, N5, 'bam_genome_local_single')
        file_sorted_bam_genome_local = create_filename(dir_sample, N7, N5, 'sorted_bam_genome_local_single')

        if not os.path.exists(os.path.dirname(file_bam_genome_local)):
            os.mkdir(os.path.dirname(file_bam_genome_local))
    
    elif single_or_paired == 'paired':
        file_sam_genome_local = create_filename(dir_sample, N7, N5, 'sam_genome_local_paired')
        file_sam_report_genome_local = create_filename(dir_sample, N7, N5, 'sam_report_genome_local_paired')
    
        if not os.path.exists(os.path.dirname(file_sam_genome_local)):
            os.mkdir(os.path.dirname(file_sam_genome_local))
    
        file_bam_genome_local = create_filename(dir_sample, N7, N5, 'bam_genome_local_paired')
        file_sorted_bam_genome_local = create_filename(dir_sample, N7, N5, 'sorted_bam_genome_local_paired')

        if not os.path.exists(os.path.dirname(file_bam_genome_local)):
            os.mkdir(os.path.dirname(file_bam_genome_local))
    
        
    # local alignment to the genome with bowtie2
    initial_dir = os.getcwd()
    
    #-X length for paired end fragment -k number of distict matches
    if single_or_paired == 'single':
        
        bowtie2_command = ['bowtie2', '--local', '-p', str(ncpu),
                           '-k', '4', '-x', assembly,
                           '-U', file_R1, '-S', file_sam_genome_local]
    elif single_or_paired == 'paired':
        bowtie2_command = ['bowtie2', '--local', '-p', str(ncpu),
                            '-X', '5000', '-k', '4', '-x', assembly,
                            '-1', file_R1, '-2', file_R2,
                            '-S', file_sam_genome_local]

    handle_sam_report_genome_local = open(file_sam_report_genome_local, 'wb')

    subprocess.call(bowtie2_command, stderr=handle_sam_report_genome_local)

    handle_sam_report_genome_local.close()

    # convert sam to bam
    sam_to_bam_genome_local_command = ['samtools', 'view', '-Sb', file_sam_genome_local]

    handle_file_bam_genome_local = open(file_bam_genome_local, 'wb')

    subprocess.call(sam_to_bam_genome_local_command, stdout=handle_file_bam_genome_local)

    # sort bam files
    sort_bam_genome_local_command = ['samtools', 'sort', file_bam_genome_local, '-o', file_sorted_bam_genome_local]

    subprocess.call(sort_bam_genome_local_command)

    # Clean up
    if keep_sam == 0:
        os.remove(file_sam_genome_local)
        print('sam file deleted')

    
    os.remove(file_bam_genome_local)

    # Create bam index files
    create_bam_genome_local_index_command = ['samtools', 'index', file_sorted_bam_genome_local]
    subprocess.call(create_bam_genome_local_index_command)

    os.chdir(initial_dir)




In [30]:

def guideseq_filtered_mapq_AS_primary(dir_sample, global_or_local, amplicon_info, single_or_paired = 'single', min_MAPQ = 25, min_AS=-180):
    #function beginning
    # define samples
    N7 = amplicon_info['index_I1']
    N5 = amplicon_info['index_I2']
    name = amplicon_info['name']


    
    if global_or_local == 'global':
        if single_or_paired == 'single':
            bam_file = create_filename(dir_sample, N7, N5, 'sorted_bam_genome_global_single')
            final_trimmed_bam_filtered_file = create_filename(dir_sample, N7, N5, 'filtered_and_sorted_genome_global_single')
        elif single_or_paired == 'paired':
            bam_file = create_filename(dir_sample, N7, N5, 'sorted_bam_genome_global_paired')
            final_trimmed_bam_filtered_file = create_filename(dir_sample, N7, N5, 'filtered_and_sorted_genome_global_paired')
 
    if global_or_local == 'local':
        if single_or_paired == 'single':
            bam_file = create_filename(dir_sample, N7, N5, 'sorted_bam_genome_local_single')
            final_trimmed_bam_filtered_file = create_filename(dir_sample, N7, N5, 'filtered_and_sorted_genome_local_single')
        elif single_or_paired == 'paired':
            print('wooo')
            bam_file = create_filename(dir_sample, N7, N5, 'sorted_bam_genome_local_paired')
            final_trimmed_bam_filtered_file = create_filename(dir_sample, N7, N5, 'filtered_and_sorted_genome_local_paired')    
    
    all_reads_unfiltered = []
    all_reads_pass_filter = []
    print(bam_file)
    bam_alignment_file = pysam.AlignmentFile(bam_file, 'rb')
    bam_in = bam_alignment_file.fetch()
    
    #make output file
    filtered_reads = pysam.AlignmentFile(final_trimmed_bam_filtered_file, "wb", template=bam_alignment_file)
    
    #test each read
    for read in bam_in:
        all_reads_unfiltered.append(read.query_name)
        if read.has_tag('AS'):
            read_AS = read.get_tag('AS')
        #check read quality is good
        if read.mapping_quality >= min_MAPQ and read_AS >= min_AS and not read.is_secondary:
            all_reads_pass_filter.append(read.query_name)
            filtered_reads.write(read)
    filtered_reads.close()
    bam_alignment_file.close()


    count_unfiltered = len(all_reads_unfiltered)

    count_filtered = len(all_reads_pass_filter)
    
    
    
    results_df = pd.DataFrame({'sample_name': [name],
                               'i7': [N7],
                               'i5': [N5],
                               'single_or_paired_reads' : [single_or_paired],
                               'all_alignments_count': [count_unfiltered],
                               'filtered_alignments_count': [count_filtered]
                               },
                                  columns=['sample_name',
                                           'i7',
                                           'i5',
                                           'single_or_paired_reads',
                                           'all_alignments_count',
                                           'filtered_alignments_count'])
    print('single por paired', single_or_paired)
    
    print(results_df)   
    return results_df



In [36]:
###  THIS IS FOR BLASTING THE LOCAL ALIGNMENT. WE WILL HAVE LIGATED AAV ON END OF READ AND SO WE SHOULD IGNORE THIS
#
#     This section does two things. It makes a bam file that is filtered for AS and MAPQ 
#     It quantifies these bam files and sorts them. Then it converts to a bed file.
#

                #### Required Arguments
cpu = 12
min_MAPQ = 50
min_AS = -180


                        #this does the acutal alignment for end-to-end
do_align = 'on'
single_or_paired_reads = "paired"     #can be 'single' or 'paired' for the read alignment


                    #### WHAT SAMPLES TO RUN ####
#lines_to_run = [15]
lines_to_run = range(15)

                    #for exporting summary of data
results_df_all = pd.DataFrame()
results_file = os.path.join(directory, 'results','local_alignment_filtered_summary.xlsx')


for i in lines_to_run:
    
    amplicon_info = get_csv_data(directory, i)
    print('running row', i)
    print('the analysis is on AAV-seq sample:', amplicon_info['name'])
    reference_assembly = amplicon_info['genome']
    sample_name = amplicon_info['name']
    
    if do_align == 'on':
        #####this aligns everything after the break to end-to-end keep_sam=1 means keep the sam file
        align_guideseq_genome_local(directory, amplicon_info, reference_assembly, single_or_paired_reads, cpu, keep_sam=1)


    #### Process the alignment files to make a bam file of only high quality reads

    #this filters for good alignments based on AS score and MapQ and exports a final indexed bam file.

    df_sample_results = guideseq_filtered_mapq_AS_primary(directory, 'local', amplicon_info, single_or_paired_reads)
    #this is a running dataframe of the clenaed up read summary
    results_df_all = results_df_all.append(df_sample_results, ignore_index=True)


    ### global alignment bed generation 
    N7 = amplicon_info['index_I1']
    N5 = amplicon_info['index_I2']
    print('-----  now making local bed file for', sample_name, ' ',N7, ' ', N5 )


    ##### Need to fix this!!!!!!!!!!!! the names of the input are not right and mabye i need to sort things
    if single_or_paired_reads == "single":
        genome_local_bed_file = create_filename(directory, N7, N5, 'guideseq_local_bed_single')
        filtered_and_sorted_bam_file = create_filename(directory, N7, N5, 'filtered_and_sorted_genome_local_single')
    elif single_or_paired_reads == "paired":
        genome_local_bed_file = create_filename(directory, N7, N5, 'guideseq_local_bed_paired')
        filtered_and_sorted_bam_file = create_filename(directory, N7, N5, 'filtered_and_sorted_genome_local_paired')

    !bam2bed < {filtered_and_sorted_bam_file} > {genome_local_bed_file}
    bed_folder = os.path.join(directory, 'bed_files')

    if not os.path.exists(bed_folder):
        os.mkdir(bed_folder)

    file_location = create_filename(directory, N7, N5, 'bam_local')

    new_bed_name = os.path.join(file_location, sample_name + '_' + 'local_'+ single_or_paired_reads + '.bed')

    os.rename(genome_local_bed_file, new_bed_name)

    shutil.copy(new_bed_name, bed_folder)




    print('made local bed file:', N7, '_', N5 )
    print('It is called', sample_name )
    print('doing the quantification of local alignments')

 
    
 
 ## export all the read summaries
results_df_all.to_excel(results_file)    


running row 0
the analysis is on AAV-seq sample: AAV_seq_1.1
wooo
/home/eric/Data/Spaced_Nicking/AAVseq1_200930_MN00157_0064_A000H37GLC/N701_S502/bam_genome_local_files/N701_S502_paired.sorted.bam
single por paired paired
   sample_name    i7    i5 single_or_paired_reads  all_alignments_count  \
0  AAV_seq_1.1  N701  S502                 paired                    17   

   filtered_alignments_count  
0                          6  
-----  now making local bed file for AAV_seq_1.1   N701   S502
made local bed file: N701 _ S502
It is called AAV_seq_1.1
doing the quantification of local alignments
running row 1
the analysis is on AAV-seq sample: AAV_seq_1.2
wooo
/home/eric/Data/Spaced_Nicking/AAVseq1_200930_MN00157_0064_A000H37GLC/N701_S503/bam_genome_local_files/N701_S503_paired.sorted.bam
single por paired paired
   sample_name    i7    i5 single_or_paired_reads  all_alignments_count  \
0  AAV_seq_1.2  N701  S503                 paired                     6   

   filtered_alignments_cou

made local bed file: N705 _ S503
It is called AAV_seq_5.2
doing the quantification of local alignments
running row 14
the analysis is on AAV-seq sample: AAV_seq_5.3
wooo
/home/eric/Data/Spaced_Nicking/AAVseq1_200930_MN00157_0064_A000H37GLC/N705_S505/bam_genome_local_files/N705_S505_paired.sorted.bam
single por paired paired
   sample_name    i7    i5 single_or_paired_reads  all_alignments_count  \
0  AAV_seq_5.3  N705  S505                 paired                     2   

   filtered_alignments_count  
0                          2  
-----  now making local bed file for AAV_seq_5.3   N705   S505
made local bed file: N705 _ S505
It is called AAV_seq_5.3
doing the quantification of local alignments


In [33]:
###  ALIGN END-TO-END GLOBAL
#s. It makes a bam file that is filtered for AS and MAPQ 
#   It quantifies these bam files and sorts them. Then it converts to a bed file.
#

                #### Required Arguments
cpu = 12
min_MAPQ = 50
min_AS = -180

             ### WHAT TYPE OF ANALYSIS ARE WE RUNNING #####


            #### this does the acutal alignment for end-to-end
do_align = 'off'
single_or_paired_reads = "paired"   #can be 'single' or 'paired' for the read alignment

            #### WHAT SAMPLES TO RUN ####
#lines_to_run = [15]
lines_to_run = range(15)

    #for exporting summary of data
results_df_all = pd.DataFrame()
results_file = os.path.join(directory, 'results','global_alignment_filtered_summary.xlsx')


for i in lines_to_run:
    
    amplicon_info = get_csv_data(directory, i)
    print('running row', i)
    print('the analysis is on AAV-seq sample:', amplicon_info['name'])
    reference_assembly = amplicon_info['genome']
    sample_name = amplicon_info['name']
    
    if do_align == 'on':
        #####this aligns everything after the break to end-to-end keep_sam=1 means keep the sam file
        align_guideseq_end_to_end_genome_global(directory, amplicon_info, reference_assembly, single_or_paired_reads, cpu, keep_sam=1)


    #### Process the alignment files to make a bam file of only high quality reads

    #this filters for good alignments based on AS score and MapQ and exports a final indexed bam file.

    df_sample_results = guideseq_filtered_mapq_AS_primary(directory, 'global', amplicon_info, single_or_paired_reads)
    #this is a running dataframe of the clenaed up read summary
    results_df_all = results_df_all.append(df_sample_results, ignore_index=True)


    ### global alignment bed generation 
    N7 = amplicon_info['index_I1']
    N5 = amplicon_info['index_I2']
    print('-----  now making bed file for', sample_name, ' ',N7, ' ', N5 )


    ##### Need to fix this!!!!!!!!!!!! the names of the input are not right and mabye i need to sort things
    if single_or_paired_reads == "single":
        genome_global_bed_file = create_filename(directory, N7, N5, 'guideseq_global_bed_single')
        filtered_and_sorted_bam_file = create_filename(directory, N7, N5, 'filtered_and_sorted_genome_global_single')
    elif single_or_paired_reads == "paired":
        genome_global_bed_file = create_filename(directory, N7, N5, 'guideseq_global_bed_paired')
        filtered_and_sorted_bam_file = create_filename(directory, N7, N5, 'filtered_and_sorted_genome_global_paired')

    !bam2bed < {filtered_and_sorted_bam_file} > {genome_global_bed_file}
    bed_folder = os.path.join(directory, 'bed_files')

    if not os.path.exists(bed_folder):
        os.mkdir(bed_folder)

    file_location = create_filename(directory, N7, N5, 'bam_global')

    new_bed_name = os.path.join(file_location, sample_name + '_' + 'global_'+ single_or_paired_reads + '.bed')

    os.rename(genome_global_bed_file, new_bed_name)

    shutil.copy(new_bed_name, bed_folder)




    print('made global bed file:', N7, '_', N5 )
    print('It is called',sample_name )
    print('doing the quantification of global alignments')


 ## export all the read summaries    
#results_df_all.to_excel(results_file)    
print(results_file)


running row 0
the analysis is on AAV-seq sample: AAV_seq_1.1
/home/eric/Data/Spaced_Nicking/AAVseq1_200930_MN00157_0064_A000H37GLC/N701_S502/bam_genome_global_files/N701_S502_paired.sorted.bam
single por paired paired
   sample_name    i7    i5 single_or_paired_reads  all_alignments_count  \
0  AAV_seq_1.1  N701  S502                 paired                    10   

   filtered_alignments_count  
0                          6  
-----  now making bed file for AAV_seq_1.1   N701   S502
made global bed file: N701 _ S502
It is called AAV_seq_1.1
doing the quantification of global alignments
running row 1
the analysis is on AAV-seq sample: AAV_seq_1.2
/home/eric/Data/Spaced_Nicking/AAVseq1_200930_MN00157_0064_A000H37GLC/N701_S503/bam_genome_global_files/N701_S503_paired.sorted.bam
single por paired paired
   sample_name    i7    i5 single_or_paired_reads  all_alignments_count  \
0  AAV_seq_1.2  N701  S503                 paired                     0   

   filtered_alignments_count  
0      

made global bed file: N705 _ S503
It is called AAV_seq_5.2
doing the quantification of global alignments
running row 14
the analysis is on AAV-seq sample: AAV_seq_5.3
/home/eric/Data/Spaced_Nicking/AAVseq1_200930_MN00157_0064_A000H37GLC/N705_S505/bam_genome_global_files/N705_S505_paired.sorted.bam
single por paired paired
   sample_name    i7    i5 single_or_paired_reads  all_alignments_count  \
0  AAV_seq_5.3  N705  S505                 paired                     2   

   filtered_alignments_count  
0                          2  
-----  now making bed file for AAV_seq_5.3   N705   S505
made global bed file: N705 _ S505
It is called AAV_seq_5.3
doing the quantification of global alignments
/home/eric/Data/Spaced_Nicking/AAVseq1_200930_MN00157_0064_A000H37GLC/results/global_alignment_filtered_summary.xlsx


In [72]:
############ Use CIRPSR Gold and look at overlaps

single_or_paired_reads = "paired"  #can be 'single' or 'paired' for the read alignment
windowsize = 500

#### WHAT SAMPLES TO RUN ####
lines_to_run = [1]
#lines_to_run = range(30)


#off_target_sites 
crispr_gold_file = '/home/eric/Data/Spaced_Nicking/GuideSeq_MiniSeq1/CrispRgold_predictions.xlsx'
df_targets = pd.read_excel(crispr_gold_file, index_col=0)

# new data frame with split guide location. CRISPRGOLD puts it all in at once
new = df_targets["Position"].str.split(":", expand = True) 
df_targets["chr"]= new[0] 
df_targets["position_start"]= new[1] 
df_targets["position_end"]= new[2] 
df_targets.drop(columns =["Position"], inplace = True) 
df_targets['position_start'] = df_targets['position_start'].astype(int)
df_targets['begin_site_range'] = df_targets['position_start'] - windowsize
df_targets['end_site_range'] = df_targets['position_start'] + windowsize

print(df_targets.dtypes)

#print(df_targets)


for i in lines_to_run:
    
    amplicon_info = get_csv_data(directory, i)
    name = amplicon_info['name']
    print('running row', i)
    print('the analysis is on Guideseq sample:', name)
    print('index',)
    
    #import bed file
    N7 = amplicon_info['index_I1']
    N5 = amplicon_info['index_I2']
    print('index', N7, N5)



    
    print(results_file)

    file_location = create_filename(directory, N7, N5, 'bam_global')   
    print(file_location)
    sample_name = amplicon_info['name']
    bedfile = os.path.join(file_location, sample_name + '_' + single_or_paired_reads + '.bed')
    print(bedfile)
    
    #needs to add header names to specify the number of columns so that 
    
    header_names = ['chr', 'chromStart', 'chromEnd', 'name', 'MapQ', 'strand', 'flag', 'CIGAR', 'paired_name', 'read1start', 'paired_amplicon_size', 'sequence', 'quality_score','a','b','c','d','e','f','g','h','i']

#    df_bed = pd.read_csv(bedfile, sep='\t', comment='t', header=None)
    df_bed = pd.read_csv(bedfile, sep='\t', header=None, names = header_names)

    
    
    #header = ['chr', 'chromStart', 'chromEnd', 'name', 'MapQ', 'strand', 'flag', 'CIGAR', 'paired_name', 'read1start', 'paired_amplicon_size', 'sequence', 'quality_score','a','b','c','d','e','f','g','h','i']
    #df_bed.columns = header[:len(df_bed.columns)]
    
    print(df_bed.dtypes) 
    
    read_hits = []
    count = 0
    for i,g in itertools.izip(df_bed.chromStart, df_bed.chr):
        bed_alignment = i
        chromosome = g
        for a,b,c,d in itertools.izip(df_targets.listnumber2, df_targets.begin_site_range, df_targets.end_site_range, df_targets.chr): 
            counter_A = 0
            if bed_alignment in range(b, c) and chromosome == d:
                if counter = 0:
                    print(b,c,'chromosome', d, ' guide:', a)
                    read_hits.append(a)
                    counter_A += 1
                    print(b,c,'chromosome', d, ' guide:', a, 'for read starting at :', i)
        count += 1 
        if count >30:
            break
    results_df = pd.DataFrame({'sample_name': [name],
                               'i7': [N7],
                               'i5': [N5],
                               'list_of_read_guide_matches' : [read_hits],
                               },
                                  columns=['sample_name',
                                           'i7',
                                           'i5',
                                           'list_of_read_guide_matches'])            
    
    
    
    results_file = os.path.join(directory, 'results',N7 + '_' + N5 + '_'+ name +'_guide_count.xlsx')
    
    results_df.to_excel(results_file)    

    print('list of read hits:',read_hits)


listnumber2                 object
Sequence                    object
Name                        object
Top off-target sites        object
Strand                      object
MutDist                      int64
Risk                        object
Annotation (if relevant)    object
chr                         object
position_start               int64
position_end                object
begin_site_range             int64
end_site_range               int64
dtype: object
running row 1
the analysis is on Guideseq sample: 2-1-3'GSP-TD
index
index N702 S502
/home/eric/Data/Spaced_Nicking/GuideSeq_MiniSeq1/results/N702_S502_2-1-3'GSP-TD_guide_count.xlsx
/home/eric/Data/Spaced_Nicking/GuideSeq_MiniSeq1/N702_S502/bam_genome_global_files
/home/eric/Data/Spaced_Nicking/GuideSeq_MiniSeq1/N702_S502/bam_genome_global_files/2-1-3'GSP-TD_paired.bed
chr                     object
chromStart               int64
chromEnd                 int64
name                    object
MapQ                     int64
stra

In [34]:

def create_filename(dir_sample, N7, N5, filetype):
    main_folder = os.path.join(dir_sample, N7 + '_' + N5)
    if filetype == 'mainfolder':
        return main_folder
    elif filetype == 'amplicons':
        return os.path.join(main_folder, 'amplicons')
    elif filetype == 'bam_global':
        return os.path.join(main_folder, 'bam_genome_global_files')
    elif filetype == 'bam_local':
        return os.path.join(main_folder, 'bam_genome_local_files')
    
    elif filetype == 'R1fastq':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R1.fastq')
    elif filetype == 'R1fastqgz':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R1.fastq.gz')
    elif filetype == 'R2fastq':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R2.fastq')
    elif filetype == 'R2fastqgz':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R2.fastq.gz')
      
    #### I added these as the sequences that were correctly primed
    elif filetype == 'R1fastq_CorrPrime':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R1.CorrPrime.fastq')
    elif filetype == 'R1fastqgz_CorrPrime':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R1.CorrPrime.fastq.gz')
    elif filetype == 'R2fastq_CorrPrime':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R2.CorrPrime.fastq')
    elif filetype == 'R2fastqgz_CorrPrime':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R2.CorrPrime.fastq.gz')
    elif filetype == 'localpriming_data':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + 'priming.xlsx')
    elif filetype == 'priming_summary':
        return os.path.join(main_folder, 'results', 'priming_summary.xlsx')
    #####
   
    
    elif filetype == 'R1trimmed':
        return os.path.join(main_folder, 'cutadapt_files', N7 + '_' + N5 + '_R1.trimmed.fastq.gz')
    elif filetype == 'R2trimmed':
        return os.path.join(main_folder, 'cutadapt_files', N7 + '_' + N5 + '_R2.trimmed.fastq.gz')
    elif filetype == 'trimmed_report':
        return os.path.join(main_folder, 'cutadapt_files', N7 + '_' + N5 + '.trimmed.report.txt')
    elif filetype == 'sam_genome_local':
        return os.path.join(main_folder, 'sam_genome_local_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'sam_report_genome_local':
        return os.path.join(main_folder, 'sam_genome_local_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'bam_genome_local':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'sorted_bam_genome_local':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'sorted_bai_genome_local':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'sam_plasmid_local':
        return os.path.join(main_folder, 'sam_plasmid_local_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'sam_report_plasmid_local':
        return os.path.join(main_folder, 'sam_plasmid_local_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'bam_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'sorted_bam_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'sorted_bai_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'unmapped_bam_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '_unmapped.bam')
    elif filetype == 'qsorted_unmapped_bam_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '_qsorted_unmapped.bam')
    elif filetype == 'unmapped_plasmid_R1fastq':
        return os.path.join(main_folder, 'plasmid_unmapped_fastq_files', N7 + '_' + N5 + '_plasmid_unmapped_R1.fastq')
    elif filetype == 'unmapped_plasmid_R2fastq':
        return os.path.join(main_folder, 'plasmid_unmapped_fastq_files', N7 + '_' + N5 + '_plasmid_unmapped_R2.fastq')
    elif filetype == 'unmapped_plasmid_R1fastqgz':
        return os.path.join(main_folder, 'plasmid_unmapped_fastq_files', N7 + '_' + N5 + '_plasmid_unmapped_R1.fastq.gz')
    elif filetype == 'unmapped_plasmid_R2fastqgz':
        return os.path.join(main_folder, 'plasmid_unmapped_fastq_files', N7 + '_' + N5 + '_plasmid_unmapped_R2.fastq.gz')
    elif filetype == 'sam_amplicons':
        return os.path.join(main_folder, 'sam_amplicon_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'sam_report_amplicons':
        return os.path.join(main_folder, 'sam_amplicon_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'bam_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'sorted_bam_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'sorted_bai_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'unmapped_bam_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '_amplicons_unmapped.bam')
    elif filetype == 'qsorted_unmapped_bam_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '_qsorted_amplicons_unmapped.bam')
    elif filetype == 'unmapped_amplicons_R1fastq':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files', N7 + '_' + N5 + '_amplicons_unmapped_R1.fastq')
    elif filetype == 'unmapped_amplicons_R2fastq':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files', N7 + '_' + N5 + '_amplicons_unmapped_R2.fastq')
    elif filetype == 'unmapped_amplicons_R1fastqgz':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files',
                            N7 + '_' + N5 + '_amplicons_unmapped_R1.fastq.gz')
    elif filetype == 'unmapped_amplicons_R2fastqgz':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files',
                            N7 + '_' + N5 + '_amplicons_unmapped_R2.fastq.gz')
    elif filetype == 'unmapped_amplicons_report':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files', N7 + '_' + N5 + '.unmapped.report.txt')
    elif filetype == 'sam_genome_global':
        return os.path.join(main_folder, 'sam_genome_global_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'sam_report_genome_global':
        return os.path.join(main_folder, 'sam_genome_global_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'bam_genome_global':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'sorted_bam_genome_global':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'sorted_bai_genome_global':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'results_amplicons':
        return os.path.join(main_folder, 'results', N7 + '_' + N5)  # We will append the window size later
    elif filetype == 'results_plasmid':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_plasmid.xlsx')
    elif filetype == 'results_all_amplicons':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_all_amplicons.xlsx')
    elif filetype == 'results_genomewide':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_genomewide.xlsx')
    elif filetype == 'summary_all_alignments':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_summary_all_alignments.xlsx')
    elif filetype == 'read_counts':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_read_counts.xlsx')
    ## more added
    elif filetype == 'results_pipeline2_global':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_pipeline2_global.xlsx')
    elif filetype == 'results_pipeline2_local':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_pipeline2_local.xlsx')
    elif filetype == 'R1trimmed':
        return os.path.join(main_folder, 'cutadapt_files', N7 + '_' + N5 + '_R1.trimmed.fastq.gz')
    elif filetype == 'R2trimmed':
        return os.path.join(main_folder, 'cutadapt_files', N7 + '_' + N5 + '_R2.trimmed.fastq.gz')
    elif filetype == 'trimmed_report':
        return os.path.join(main_folder, 'cutadapt_files', N7 + '_' + N5 + '.trimmed.report.txt')
    elif filetype == 'sam_genome_local':
        return os.path.join(main_folder, 'sam_genome_local_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'sam_report_genome_local':
        return os.path.join(main_folder, 'sam_genome_local_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'bam_genome_local':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'sorted_bam_genome_local':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'sorted_bai_genome_local':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'sam_plasmid_local':
        return os.path.join(main_folder, 'sam_plasmid_local_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'sam_report_plasmid_local':
        return os.path.join(main_folder, 'sam_plasmid_local_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'bam_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'sorted_bam_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'sorted_bai_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'unmapped_bam_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '_unmapped.bam')
    elif filetype == 'qsorted_unmapped_bam_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '_qsorted_unmapped.bam')
    elif filetype == 'unmapped_plasmid_R1fastq':
        return os.path.join(main_folder, 'plasmid_unmapped_fastq_files', N7 + '_' + N5 + '_plasmid_unmapped_R1.fastq')
    elif filetype == 'unmapped_plasmid_R2fastq':
        return os.path.join(main_folder, 'plasmid_unmapped_fastq_files', N7 + '_' + N5 + '_plasmid_unmapped_R2.fastq')
    elif filetype == 'unmapped_plasmid_R1fastqgz':
        return os.path.join(main_folder, 'plasmid_unmapped_fastq_files', N7 + '_' + N5 + '_plasmid_unmapped_R1.fastq.gz')
    elif filetype == 'unmapped_plasmid_R2fastqgz':
        return os.path.join(main_folder, 'plasmid_unmapped_fastq_files', N7 + '_' + N5 + '_plasmid_unmapped_R2.fastq.gz')
    elif filetype == 'sam_amplicons':
        return os.path.join(main_folder, 'sam_amplicon_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'sam_report_amplicons':
        return os.path.join(main_folder, 'sam_amplicon_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'bam_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'sorted_bam_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'sorted_bai_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'unmapped_bam_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '_amplicons_unmapped.bam')
    elif filetype == 'qsorted_unmapped_bam_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '_qsorted_amplicons_unmapped.bam')
    elif filetype == 'unmapped_amplicons_R1fastq':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files', N7 + '_' + N5 + '_amplicons_unmapped_R1.fastq')
    elif filetype == 'unmapped_amplicons_R2fastq':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files', N7 + '_' + N5 + '_amplicons_unmapped_R2.fastq')
    elif filetype == 'unmapped_amplicons_R1fastqgz':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files',
                            N7 + '_' + N5 + '_amplicons_unmapped_R1.fastq.gz')
    elif filetype == 'unmapped_amplicons_R2fastqgz':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files',
                            N7 + '_' + N5 + '_amplicons_unmapped_R2.fastq.gz')
    elif filetype == 'unmapped_amplicons_report':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files', N7 + '_' + N5 + '.unmapped.report.txt')
    elif filetype == 'sam_genome_global':
        return os.path.join(main_folder, 'sam_genome_global_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'sam_report_genome_global':
        return os.path.join(main_folder, 'sam_genome_global_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'bam_genome_global':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'sorted_bam_genome_global':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'sorted_bai_genome_global':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'results_amplicons':
        return os.path.join(main_folder, 'results', N7 + '_' + N5)  # We will append the window size later
    elif filetype == 'results_plasmid':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_plasmid.xlsx')
    elif filetype == 'results_all_amplicons':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_all_amplicons.xlsx')
    elif filetype == 'results_genomewide':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_genomewide.xlsx')
    elif filetype == 'summary_all_alignments':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_summary_all_alignments.xlsx')
    elif filetype == 'read_counts':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_read_counts.xlsx')
    ## more added
    elif filetype == 'results_pipeline2_global':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_pipeline2_global.xlsx')
    elif filetype == 'results_pipeline2_local':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_pipeline2_local.xlsx')
    elif filetype == 'sam_genome_global_single':
        return os.path.join(main_folder, 'sam_genome_global_files', N7 + '_' + N5 + '_single.sam')
    elif filetype == 'sam_genome_local_single':
        return os.path.join(main_folder, 'sam_genome_local_files', N7 + '_' + N5 + '_single.sam')
    elif filetype == 'sam_report_genome_global_single':
        return os.path.join(main_folder, 'sam_genome_global_files', N7 + '_' + N5 + '_single.sam.report.txt')
    elif filetype == 'sam_report_genome_local_single':
        return os.path.join(main_folder, 'sam_genome_local_files', N7 + '_' + N5 + '_single.sam.report.txt')
    elif filetype == 'bam_genome_global_single':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '_single.bam')
    elif filetype == 'bam_genome_local_single':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '_single.bam')
    elif filetype == 'sorted_bam_genome_global_single':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '_single.sorted.bam')
    elif filetype == 'sorted_bam_genome_local_single':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '_single.sorted.bam')
    elif filetype == 'sam_genome_global_paired':
        return os.path.join(main_folder, 'sam_genome_global_files', N7 + '_' + N5 + '_paired.sam')
    elif filetype == 'sam_genome_local_paired':
        return os.path.join(main_folder, 'sam_genome_local_files', N7 + '_' + N5 + '_paired.sam')
    elif filetype == 'sam_report_genome_global_paired':
        return os.path.join(main_folder, 'sam_genome_global_files', N7 + '_' + N5 + '_paired.sam.report.txt')
    elif filetype == 'sam_report_genome_local_paired':
        return os.path.join(main_folder, 'sam_genome_local_files', N7 + '_' + N5 + '_paired.sam.report.txt')
    elif filetype == 'bam_genome_global_paired':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '_paired.bam')
    elif filetype == 'bam_genome_local_paired':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '_paired.bam')
    elif filetype == 'sorted_bam_genome_global_paired':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '_paired.sorted.bam')
    elif filetype == 'sorted_bam_genome_local_paired':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '_paired.sorted.bam')
    elif filetype == 'filtered_and_sorted_genome_global_single':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '_single_primary_as_mapq.sorted.bam')    
    elif filetype == 'filtered_and_sorted_genome_local_single':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '_single_primary_as_mapq.sorted.bam')    
    elif filetype == 'genome_global_bed_single':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '_single_global.sorted.bed')
    elif filetype == 'filtered_and_sorted_genome_global_paired':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '_paired_primary_as_mapq.sorted.bam')  
    elif filetype == 'filtered_and_sorted_genome_local_paired':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '_paired_primary_as_mapq.sorted.bam')  
    elif filetype == 'genome_global_bed_paired':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '_paired_global.sorted.bed')
    elif filetype == 'guideseq_global_bed_single':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '_single.global.sorted.bed')
    elif filetype == 'guideseq_global_bed_paired':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '_paired.global.sorted.bed')
    elif filetype == 'guideseq_local_bed_single':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '_single.local.sorted.bed')
    elif filetype == 'guideseq_local_bed_paired':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '_paired.local.sorted.bed')
###########################
#

In [22]:
print(range(1,5))

[1, 2, 3, 4]
